In [2]:
!pip install transformers

In [3]:
import tensorflow as tf
# import keras
tf.__version__

'2.1.0'

In [4]:
# keras.__version__

In [5]:
# Utilities

import os, zipfile, sys, warnings
os.environ['TF_KERAS'] = '1'
from time import time

# Numerical calculation
import numpy as np

# Data Handling
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
# from keras.layers import *
from keras.utils.np_utils import to_categorical
from keras.initializers import Constant
import re


# Data Visualization
# import matplotlib.pyplot as plt
# %matplotlib inline

# import seaborn as sns

# Sequential Modeling
# import keras.backend as K
# from keras import initializers
# from keras.engine.topology import Layer, InputSpec
# from keras import Model, Sequential
# from tensorflow.keras.layers import Dense, LSTM, TimeDistributed, Conv1D, MaxPooling1D
# from tensorflow.keras.layers import Embedding, Activation, Dropout, Flatten, Bidirectional, Input
from tensorflow.keras.layers import *
# from tensorflow.keras.layers import Permute, merge, Input, multiply, concatenate
# from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
# from keras.constraints import max_norm, unit_norm

# Preprocessing and NLP library
# from keras.preprocessing.text import Tokenizer, text_to_word_sequence
# from keras.preprocessing.sequence import pad_sequences
# import nltk

# Evaluation metrics
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, accuracy_score, precision_recall_curve

Using TensorFlow backend.


In [6]:
def _get_segments(sentences):
    sentences_segments = []
    for sent in sentences:
      temp = []
      i = 0
      for token in sent.split(" "):
        temp.append(i)
        if token == "[SEP]":
          i += 1
      sentences_segments.append(temp)
    return sentences_segments

def _get_inputs(df,_maxlen,tokenizer,use_keras_pad=False):


    maxqnans = np.int((_maxlen-20)/2)
    pattern = '[^\w\s]+|\n' # remove everything including newline (|\n) other than words (\w) or spaces (\s)
    
    sentences = ["[CLS] " + " ".join(tokenizer.tokenize(desc)[:maxqnans]) +" [SEP] " 
                for desc in df['Clean Description'].values.tolist()]
              #train.head()[['question_title','question_body','answer','category']].values.tolist()]
    

    #generate masks
    # bert requires a mask for the words which are padded. 
    # Say for example, maxlen is 100, sentence size is 90. then, [1]*90 + [0]*[100-90]
    sentences_mask = [[1]*len(sent.split(" "))+[0]*(_maxlen - len(sent.split(" "))) for sent in sentences]
 
    #generate input ids  
    # if less than max length provided then the words are padded
    if use_keras_pad:
      sentences_padded = pad_sequences(sentences.split(" "), dtype=object, maxlen=10, value='[PAD]',padding='post')
    else:
      sentences_padded = [sent + " [PAD]"*(_maxlen-len(sent.split(" "))) if len(sent.split(" "))!=_maxlen else sent for sent in sentences ]

    sentences_converted = [tokenizer.convert_tokens_to_ids(s.split(" ")) for s in sentences_padded]
    
    #generate segments
    # for each separation [SEP], a new segment is converted
    sentences_segment = _get_segments(sentences_padded)

    genLength = set([len(sent.split(" ")) for sent in sentences_padded])

    if _maxlen < 20:
      raise Exception("max length cannot be less than 20")
    elif len(genLength)!=1: 
      print(genLength)
      raise Exception("sentences are not of same size")



    #convert list into tensor integer arrays and return it
    #return sentences_converted,sentences_segment, sentences_mask
    #return [np.asarray(sentences_converted, dtype=np.int32), 
    #        np.asarray(sentences_segment, dtype=np.int32), 
    #        np.asarray(sentences_mask, dtype=np.int32)]
    return [tf.cast(sentences_converted,tf.int32), tf.cast(sentences_segment,tf.int32), tf.cast(sentences_mask,tf.int32)]

In [7]:
from transformers import *

In [8]:
bert_tokenizer_transformer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [9]:
df=pd.read_csv('model_df1.csv')
df.head()

,Assignment group,Clean Description
0,0,login issue verify user detail employee manage...
1,0,outlook team meeting skype etc not appear cale...
2,0,can not log vpn not best
3,0,unable access hr tool page
4,0,skype error


In [10]:
X_train, X_test, y_train, y_test = train_test_split(df['Clean Description'], df['Assignment group'], test_size=0.20, random_state=1, stratify=df['Assignment group'])


In [11]:
y_train.shape

(6800,)

In [12]:
output_class=y_test.nunique()
output_class

36

In [13]:
df_tr=pd.DataFrame(X_train)
df_tr.columns = ['Clean Description']

In [14]:
df_ts=pd.DataFrame(X_test)
df_ts.columns = ['Clean Description']

In [15]:
X_test_tok=_get_inputs(df=df_ts,tokenizer=bert_tokenizer_transformer,_maxlen=100)

In [16]:
X_train_tok=_get_inputs(df=df_tr,tokenizer=bert_tokenizer_transformer,_maxlen=100)

In [17]:
X_train_tok[2]

<tf.Tensor: shape=(6800, 100), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>

In [18]:
MAX_SEQUENCE_LENGTH=100

In [19]:
# # token_inputs = Input(shape=(None,), name='word_inputs', dtype='int32')
# # mask_inputs = Input(shape=(None,), name='word_inputs', dtype='int32')
# # seg_inputs = Input(shape=(None,), name='word_inputs', dtype='int32')
# inputs  = Input(shape=(None, ),dtype=tf.int32)
token_inputs = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
mask_inputs = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
seg_inputs = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_segments')

In [20]:
bert_model=TFBertModel.from_pretrained("bert-base-uncased",output_hidden_states=False)


In [21]:
bert_model([token_inputs])

(<tf.Tensor 'tf_bert_model/Identity:0' shape=(None, 100, 768) dtype=float32>,
 <tf.Tensor 'tf_bert_model/Identity_1:0' shape=(None, 768) dtype=float32>)

In [22]:
seq_output,_ = bert_model([token_inputs, mask_inputs, seg_inputs])
seq_output

<tf.Tensor 'tf_bert_model_1/Identity:0' shape=(None, 100, 768) dtype=float32>

In [23]:
X = GlobalAveragePooling1D()(seq_output)
X = Dense(128, activation='relu')(X)
X = Dense(64, activation='relu')(X)
output_= Dense(36, activation='softmax', name='output')(X)

In [24]:
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import sequence

In [25]:
bert_model2 = Model([token_inputs, mask_inputs, seg_inputs],output_)
bert_model2.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_segments (InputLayer)     [(None, 100)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     ((None, 100, 768), ( 109482240   input_word_ids[0][0]             
                                                                 input_masks[0][0]            

In [26]:
y_train.nunique()

36

In [27]:
y_train_cat=to_categorical(np.array(y_train))
y_test_cat=to_categorical(np.array(y_test))

In [28]:
y_train_cat.shape

(6800, 36)

In [29]:
opt = keras.optimizers.Adam(learning_rate=2e-4)

In [30]:
bert_model2.compile(optimizer=opt, loss='categorical_crossentropy')

In [31]:
token_inputs, mask_inputs, seg_inputs

(<tf.Tensor 'input_word_ids:0' shape=(None, 100) dtype=int32>,
 <tf.Tensor 'input_masks:0' shape=(None, 100) dtype=int32>,
 <tf.Tensor 'input_segments:0' shape=(None, 100) dtype=int32>)

In [32]:
bert_model2.fit(X_train_tok,y_train_cat,epochs=1,batch_size = 18,validation_split=0.2, verbose=1)

Train on 5440 samples, validate on 1360 samples
5440/5440 [==============================] - 216s 40ms/sample - loss: 2.5039 - val_loss: 2.4374


In [33]:
bert_model2.fit(X_train_tok,y_train_cat,epochs=1,batch_size = 18,validation_split=0.2, verbose=1)

Train on 5440 samples, validate on 1360 samples
5440/5440 [==============================] - 198s 36ms/sample - loss: 2.4220 - val_loss: 2.4217


In [34]:
bert_model2.fit(X_train_tok,y_train_cat,epochs=1,batch_size = 18,validation_split=0.2, verbose=1)

Train on 5440 samples, validate on 1360 samples
5440/5440 [==============================] - 195s 36ms/sample - loss: 2.4120 - val_loss: 2.4230


In [35]:
bert_model2.fit(X_train_tok,y_train_cat,epochs=1,batch_size = 18,validation_split=0.2, verbose=1)

Train on 5440 samples, validate on 1360 samples
5440/5440 [==============================] - 199s 37ms/sample - loss: 2.4154 - val_loss: 2.4232


In [36]:
y_proba = bert_model2.predict(X_test_tok)

In [37]:
y_pred = (y_proba > 0.5).astype('int32')

In [38]:
def classification_summary(y_test, y_pred, y_proba):
    print('\033[1mModel accuracy:\033[0m %.2f%%' % (accuracy_score(y_test, y_pred) * 100))
    print('_'*80)
    print('\033[1mConfusion matrix:\033[0m\n %s' % (confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))))
    print('_'*80)
    print('\033[1mClassification report:\033[0m\n %s' % (classification_report(y_test, y_pred)))
    print('_'*80)

In [39]:
y_proba

array([[0.5067068 , 0.00375882, 0.0268866 , ..., 0.0040921 , 0.00410931,
        0.03255539],
       [0.5067068 , 0.00375882, 0.0268866 , ..., 0.0040921 , 0.00410931,
        0.03255539],
       [0.50670683, 0.00375882, 0.0268866 , ..., 0.0040921 , 0.00410931,
        0.0325554 ],
       ...,
       [0.50670683, 0.00375882, 0.02688659, ..., 0.0040921 , 0.00410931,
        0.03255539],
       [0.50670683, 0.00375882, 0.0268866 , ..., 0.0040921 , 0.00410931,
        0.03255538],
       [0.50670683, 0.00375882, 0.0268866 , ..., 0.0040921 , 0.00410931,
        0.03255538]], dtype=float32)

In [40]:
y_pred

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [41]:
classification_summary(y_test_cat, y_pred, y_proba)

Model accuracy: 46.76%
________________________________________________________________________________
Confusion matrix:
 [[795   0   0 ...   0   0   0]
 [  6   0   0 ...   0   0   0]
 [ 48   0   0 ...   0   0   0]
 ...
 [  7   0   0 ...   0   0   0]
 [  7   0   0 ...   0   0   0]
 [ 71   0   0 ...   0   0   0]]
________________________________________________________________________________
Classification report:
               precision    recall  f1-score   support

           0       0.47      1.00      0.64       795
           1       0.00      0.00      0.00         6
           2       0.00      0.00      0.00        48
           3       0.00      0.00      0.00        40
           4       0.00      0.00      0.00        20
           5       0.00      0.00      0.00        26
           6       0.00      0.00      0.00        37
           7       0.00      0.00      0.00        14
           8       0.00      0.00      0.00       132
           9       0.00      0.00      

E:\Anaconda\envs\huggingface\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
